# BMO Credit Card Use Case 2 Modelling: 1M Transactions

Bit of set up to start with...  Let's make sure skflow is installed.

In [ ]:
%%bash
pip install skflow

Imports...

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gcp
import gcp.bigquery as bq
import skflow
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

Personal preference but I like to wind these configuration settings up because we work with some wide DataFrames and I like to see all the column headings.  Similar argument for rows.

In [ ]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

## Use Case 2: Method of Payment of Card Balances

From the use case description document...

* This scenario purports to detect AML risks in which a money launder may be using money from hard-to-trace source to pay off credit card balance.
* Categorize the credit payments by the payment methods and look for unusual patterns, e.g. consistent or large usage of ACH by a domestic customer to pay off the credit card
* The following fields should be part of transaction data schema.

|Transaction Type|Credit_Debit_Flag|
|---|---|
|Payment|C (Credit)|
|Refund|C|
|Reversal|C|
|Award|C|
|Purchase|D (Debit)|
|Fee|D|
|Interest Charge|D|
|Penalty|D|

|Payment Method|
|---|
|Cash|
|Wire|
|ACH|
|Paper Check|
|e-Check|
|Online Transfer|
|Payment at ATM|

What we'll take from this is the simplest possible interpretation...  We'll look at credits made to accounts and see if we can infer fraud from the patterns in those credits.  N.B. We'll ignore all other factors in this first version although we woudn't in later versions.  This one is for illustrative purposes.

## Exploratory Data Analysis (Abbreviated)

First of all, let's get a feel for the data...

We'll pull in the data for Use Case 2 and pivot it to give us a count for each variant of the use case (Green, Yellow, Red) of the number of transactions of each credit action.

In [ ]:
%%sql --module use_case_2

DEFINE QUERY trans_by_type
SELECT
  USE_CASE as use_case,
  TRANSACTION_TYPE AS trans_type,
  COUNT(*) AS num_trans
FROM
  [CreditCardv2.cc_trans_02052016]
WHERE
  REGEXP_MATCH(USE_CASE, 'Use Case 2')
  AND CREDIT_DEBIT == 'C'
GROUP BY
  trans_type, use_case
ORDER BY
  trans_type, use_case

In [ ]:
trans_by_type = bq.Query(use_case_2.trans_by_type).to_dataframe()

In [ ]:
trans_by_type_pivoted = trans_by_type.pivot('trans_type', 'use_case')
trans_by_type_pivoted = trans_by_type_pivoted.fillna(0)
trans_by_type_pivoted

In [ ]:
_ = trans_by_type_pivoted.plot(kind='bar', figsize=(15,10))

Which effectively shows us that there are a lot more instances of Green, than Yellow, than Red.  This is physically true but doesn't advance our hypothesis so let's scale the data so everything is a fraction of 1.

In [ ]:
trans_by_type_pivoted = trans_by_type_pivoted.divide(trans_by_type_pivoted.sum(), axis=1)
trans_by_type_pivoted

In [ ]:
_ = trans_by_type_pivoted.plot(kind='bar', figsize=(15,10))

Now we see some signal in the data, good.

## Model Development

We'll stay with our theme of simplicity.  Our model will be the following:

* Sample the transaction data, take only the credit events, and take equal populations of Green, Yellow and Red.
* For each account in the sampled data and ransform the data from rows in sequential order, to columns in the same order.  I.e.
    
||||
|---|---|---|
|1|ACH|100|
|2|Check|50|
|3|Wire|25|
    
   becomes,
    
||||
|---|---|---|
|ACH|Check|Wire|

* For each row, encode the data one-hot.  I.e. with three options ACH, Check, Wire, each option is encoded as a vector of size 3, e.g.

|||||
|---|---|---|---|
|ACH|0|0|1|
|Check|0|1|0|
|Wire|1|0|0|

So, effectively we'll look at time-ordered sequences of credit actions and see if we can learn patterns from them that indicate fraud.

We'll focus on two things here:

1. The length of the time-ordered sequence.  This is super-important because the aim here is to identify fraud as early as possible.  If our model does a good job of classifying fraud given 5 years of data that's fraud that's occured for 5 years.  If our model does a good job of classifying fraud given 6-12 months of data that's a vast improvement.
1. The type of model - RandomForest versus Feed Forward Neural Net.

N.B. Challenge everything!  This is one approach to this modelling exercise.  It's a signpost in the right direction but should be regarded as incomplete.

In [ ]:
%%sql --module use_case_2

DEFINE QUERY sample_trans
SELECT
  *
FROM
  [CreditCardv2.cc_trans_02052016]
WHERE
  USE_CASE = $use_case AND
  CREDIT_DEBIT = 'C'
ORDER BY
  ACCOUNTID, TRANSACTION_DATE
LIMIT
  $sample_size

In [ ]:
SAMPLE_SIZE = 1000000

GREEN_USE_CASE = 'Use Case 2 - Green'
YELLOW_USE_CASE = 'Use Case 2 - Yellow'
RED_USE_CASE = 'Use Case 2 - Red'

GREEN = 'Green'
YELLOW = 'Yellow'
RED = 'Red'

In [ ]:
# This function transforms our DataFrame from a row per transaction to a column per credit event type.
# This introduces some NaN elements so it also transforms those to 'Missing'
# TRANSACTION_DATE is included to allow us to later combine and sort different samples together.

def widen_trans_per_account(sample_trans, use_case, width):
  
  sample_trans_acct_ids = sample_trans.ACCOUNTID.drop_duplicates()

  sample_trans_wide = pd.DataFrame()

  for sample_trans_acct_id in sample_trans_acct_ids:
    sample_trans_acct = sample_trans[sample_trans.ACCOUNTID == sample_trans_acct_id]
    tmp = pd.DataFrame()
    for i in range(width):
      tmp['shift_' + str(i)] = sample_trans_acct.TRANSACTION_TYPE.shift(i)
    sample_trans_wide = pd.concat([sample_trans_wide, tmp])

  sample_trans_wide['USE_CASE'] = use_case
  sample_trans_wide['TRANSACTION_DATE'] = sample_trans.TRANSACTION_DATE
  
  sample_trans_wide = sample_trans_wide.fillna('Missing')
  
  return sample_trans_wide

## Source the Transaction Data

In [ ]:
sample_trans_green = bq.Query(use_case_2.sample_trans, use_case = GREEN_USE_CASE, sample_size=SAMPLE_SIZE).to_dataframe()
sample_trans_green[:3]

In [ ]:
sample_trans_yellow = bq.Query(use_case_2.sample_trans, use_case = YELLOW_USE_CASE, sample_size=SAMPLE_SIZE).to_dataframe()
sample_trans_yellow[:3]

In [ ]:
sample_trans_red = bq.Query(use_case_2.sample_trans, use_case=RED_USE_CASE, sample_size=SAMPLE_SIZE).to_dataframe()
sample_trans_red[:3]

## Transform the Data to Time-Ordered Sequences and Rock and Roll V1

In [ ]:
WIDTH = 12 # Look at the last 12 payments...

In [ ]:
sample_trans_green_wide = widen_trans_per_account(sample_trans_green, GREEN, WIDTH) 

print sample_trans_green_wide.shape
print
print sample_trans_green_wide.describe()
print
sample_trans_green_wide[:3]

In [ ]:
sample_trans_yellow_wide = widen_trans_per_account(sample_trans_yellow, YELLOW, WIDTH)

print sample_trans_yellow_wide.shape
print
print sample_trans_yellow_wide.describe()
print
sample_trans_yellow_wide[:3]

In [ ]:
sample_trans_red_wide = widen_trans_per_account(sample_trans_red, RED, WIDTH)

print sample_trans_red_wide.shape
print
print sample_trans_red_wide.describe()
print
sample_trans_red_wide[:3]

In [ ]:
sample_trans_wide = pd.concat([sample_trans_green_wide, sample_trans_yellow_wide, sample_trans_red_wide])
sample_trans_wide = sample_trans_wide.sort_values('TRANSACTION_DATE')
sample_trans_wide = sample_trans_wide.drop('TRANSACTION_DATE', 1)

print sample_trans_wide.describe()
print
sample_trans_wide[:3]
#why/how did we merge all Credit types in shift_0?

In [ ]:
shift_encoder = preprocessing.LabelEncoder()
shift_encoder.fit(sample_trans_wide.shift_1.drop_duplicates())

use_case_encoder = preprocessing.LabelEncoder()
use_case_encoder.fit(sample_trans_wide.USE_CASE.drop_duplicates())

sample_trans_wide_enc = pd.DataFrame(index=sample_trans_wide.index)

sample_trans_wide_enc['USE_CASE'] = use_case_encoder.transform(sample_trans_wide.USE_CASE)

for i in range(WIDTH):
  sample_trans_wide_enc['shift_' + str(i) + '_enc'] = shift_encoder.transform(sample_trans_wide['shift_' + str(i)])

sample_trans_wide_enc[:5]
#does the encoder automatically assign values 0-6? 

In [ ]:
X = sample_trans_wide_enc[sample_trans_wide_enc.columns[1:]]
X[:5]

In [ ]:
y = sample_trans_wide_enc[sample_trans_wide_enc.columns[0]]
y[:5]

In [ ]:
X_enc = pd.DataFrame()
for i in range(WIDTH):
  enc = preprocessing.OneHotEncoder()
  enc.fit(X['shift_' + str(i) + '_enc'].values.reshape((-1,1)))
  X_enc = pd.concat([X_enc, pd.DataFrame(enc.transform(X['shift_' + str(i) + '_enc'].values.reshape((-1,1))).toarray())], axis=1)

X_enc.columns = [i for i in range(len(X_enc.columns))]
X_enc[:5]
#does this code block vectorize the encoded data frames? 

In [ ]:
X_split = int(len(X_enc) * 0.8)
X_train = X_enc[:X_split]
X_test = X_enc[X_split:]

y_split = int(len(y) * 0.8)
y_train = y[:y_split]
y_test = y[y_split:]

print X_train.shape, X_test.shape, y_train.shape, y_test.shape
#splitting training and testing datasets to 80/20

In [ ]:
classifier = RandomForestClassifier(n_estimators = 25)
#why 25 trees? 
classifier.fit(X_train, y_train)

print accuracy_score(y_train, classifier.predict(X_train))
print accuracy_score(y_test, classifier.predict(X_test))
print

y_pred = classifier.predict(X_test)
print metrics.classification_report(y_test, y_pred)
print
print metrics.confusion_matrix(y_test, y_pred)

In [ ]:
def model(X, y):
  layers = skflow.ops.dnn(X, [25], keep_prob=0.5)
  return skflow.models.logistic_regression(layers, y)

classifier = skflow.TensorFlowEstimator(
  model_fn=model, n_classes=3, steps=10000, learning_rate=0.003, optimizer='Adam')
classifier.fit(X_train, y_train)

print accuracy_score(y_train, classifier.predict(X_train))
print accuracy_score(y_test, classifier.predict(X_test))
print

y_pred = classifier.predict(X_test)
print metrics.classification_report(y_test, y_pred)
print
print metrics.confusion_matrix(y_test, y_pred)

## Transform the Data to Time-Ordered Sequences and Rock and Roll V2

In [ ]:
WIDTH = 24 # Look at the last 24 payments...

sample_trans_green_wide = widen_trans_per_account(sample_trans_green, GREEN, WIDTH)

print sample_trans_green_wide.shape
print
print sample_trans_green_wide.describe()
print
print sample_trans_green_wide[:3]
print

sample_trans_yellow_wide = widen_trans_per_account(sample_trans_yellow, YELLOW, WIDTH)

print sample_trans_yellow_wide.shape
print
print sample_trans_yellow_wide.describe()
print
print sample_trans_yellow_wide[:3]
print

sample_trans_red_wide = widen_trans_per_account(sample_trans_red, RED, WIDTH)

print sample_trans_red_wide.shape
print
print sample_trans_red_wide.describe()
print
print sample_trans_red_wide[:3]
print

sample_trans_wide = pd.concat([sample_trans_green_wide, sample_trans_yellow_wide, sample_trans_red_wide])
sample_trans_wide = sample_trans_wide.sort_values('TRANSACTION_DATE')
sample_trans_wide = sample_trans_wide.drop('TRANSACTION_DATE', 1)

print sample_trans_wide.describe()
print
print sample_trans_wide[:3]
print

shift_encoder = preprocessing.LabelEncoder()
shift_encoder.fit(sample_trans_wide.shift_1.drop_duplicates())

use_case_encoder = preprocessing.LabelEncoder()
use_case_encoder.fit(sample_trans_wide.USE_CASE.drop_duplicates())

sample_trans_wide_enc = pd.DataFrame(index=sample_trans_wide.index)

sample_trans_wide_enc['USE_CASE'] = use_case_encoder.transform(sample_trans_wide.USE_CASE)

for i in range(WIDTH):
  sample_trans_wide_enc['shift_' + str(i) + '_enc'] = shift_encoder.transform(sample_trans_wide['shift_' + str(i)])

print sample_trans_wide_enc[:5]
print 

X = sample_trans_wide_enc[sample_trans_wide_enc.columns[1:]]
print X[:5]
print 

y = sample_trans_wide_enc[sample_trans_wide_enc.columns[0]]
print y[:5]
print

X_enc = pd.DataFrame()
for i in range(WIDTH):
  enc = preprocessing.OneHotEncoder()
  enc.fit(X['shift_' + str(i) + '_enc'].values.reshape((-1,1)))
  X_enc = pd.concat([X_enc, pd.DataFrame(enc.transform(X['shift_' + str(i) + '_enc'].values.reshape((-1,1))).toarray())], axis=1)

X_enc.columns = [i for i in range(len(X_enc.columns))]
print X_enc[:5]
print

X_split = int(len(X_enc) * 0.8)
X_train = X_enc[:X_split]
X_test = X_enc[X_split:]

y_split = int(len(y) * 0.8)
y_train = y[:y_split]
y_test = y[y_split:]

print X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
classifier = RandomForestClassifier(n_estimators = 50)
classifier.fit(X_train, y_train)

print accuracy_score(y_train, classifier.predict(X_train))
print accuracy_score(y_test, classifier.predict(X_test))
print

y_pred = classifier.predict(X_test)
print metrics.classification_report(y_test, y_pred)
print
print metrics.confusion_matrix(y_test, y_pred)

In [ ]:
def model(X, y):
  layers = skflow.ops.dnn(X, [50], keep_prob=0.5)
  return skflow.models.logistic_regression(layers, y)

classifier = skflow.TensorFlowEstimator(
  model_fn=model, n_classes=3, steps=10000, learning_rate=0.003, optimizer='Adam')
classifier.fit(X_train, y_train)

print accuracy_score(y_train, classifier.predict(X_train))
print accuracy_score(y_test, classifier.predict(X_test))
print

y_pred = classifier.predict(X_test)
print metrics.classification_report(y_test, y_pred)
print
print metrics.confusion_matrix(y_test, y_pred)

## Transform the Data to Time-Ordered Sequences and Rock and Roll V3

In [ ]:
WIDTH = 60 # Look at the last 60 payments...

sample_trans_green_wide = widen_trans_per_account(sample_trans_green, GREEN, WIDTH)

print sample_trans_green_wide.shape
print
print sample_trans_green_wide.describe()
print
print sample_trans_green_wide[:3]
print

sample_trans_yellow_wide = widen_trans_per_account(sample_trans_yellow, YELLOW, WIDTH)

print sample_trans_yellow_wide.shape
print
print sample_trans_yellow_wide.describe()
print
print sample_trans_yellow_wide[:3]
print

sample_trans_red_wide = widen_trans_per_account(sample_trans_red, RED, WIDTH)

print sample_trans_red_wide.shape
print
print sample_trans_red_wide.describe()
print
print sample_trans_red_wide[:3]
print

sample_trans_wide = pd.concat([sample_trans_green_wide, sample_trans_yellow_wide, sample_trans_red_wide])
sample_trans_wide = sample_trans_wide.sort_values('TRANSACTION_DATE')
sample_trans_wide = sample_trans_wide.drop('TRANSACTION_DATE', 1)

print sample_trans_wide.describe()
print
print sample_trans_wide[:3]
print

shift_encoder = preprocessing.LabelEncoder()
shift_encoder.fit(sample_trans_wide.shift_1.drop_duplicates())

use_case_encoder = preprocessing.LabelEncoder()
use_case_encoder.fit(sample_trans_wide.USE_CASE.drop_duplicates())

sample_trans_wide_enc = pd.DataFrame(index=sample_trans_wide.index)

sample_trans_wide_enc['USE_CASE'] = use_case_encoder.transform(sample_trans_wide.USE_CASE)

for i in range(WIDTH):
  sample_trans_wide_enc['shift_' + str(i) + '_enc'] = shift_encoder.transform(sample_trans_wide['shift_' + str(i)])

print sample_trans_wide_enc[:5]
print 

X = sample_trans_wide_enc[sample_trans_wide_enc.columns[1:]]
print X[:5]
print 

y = sample_trans_wide_enc[sample_trans_wide_enc.columns[0]]
print y[:5]
print

X_enc = pd.DataFrame()
for i in range(WIDTH):
  enc = preprocessing.OneHotEncoder()
  enc.fit(X['shift_' + str(i) + '_enc'].values.reshape((-1,1)))
  X_enc = pd.concat([X_enc, pd.DataFrame(enc.transform(X['shift_' + str(i) + '_enc'].values.reshape((-1,1))).toarray())], axis=1)

X_enc.columns = [i for i in range(len(X_enc.columns))]
print X_enc[:5]
print

X_split = int(len(X_enc) * 0.8)
X_train = X_enc[:X_split]
X_test = X_enc[X_split:]

y_split = int(len(y) * 0.8)
y_train = y[:y_split]
y_test = y[y_split:]

print X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
classifier = RandomForestClassifier(n_estimators = 250)
classifier.fit(X_train, y_train)

print accuracy_score(y_train, classifier.predict(X_train))
print accuracy_score(y_test, classifier.predict(X_test))
print

y_pred = classifier.predict(X_test)
print metrics.classification_report(y_test, y_pred)
print
print metrics.confusion_matrix(y_test, y_pred)

In [ ]:
def model(X, y):
  layers = skflow.ops.dnn(X, [400], keep_prob=0.5)
  return skflow.models.logistic_regression(layers, y)

classifier = skflow.TensorFlowEstimator(
  model_fn=model, n_classes=3, steps=10000, learning_rate=0.003, optimizer='Adam')
classifier.fit(X_train, y_train)

print accuracy_score(y_train, classifier.predict(X_train))
print accuracy_score(y_test, classifier.predict(X_test))
print

y_pred = classifier.predict(X_test)
print metrics.classification_report(y_test, y_pred)
print
print metrics.confusion_matrix(y_test, y_pred)

## Transform the Data to Time-Ordered Sequences and Rock and Roll V4

In [ ]:
WIDTH = 100 # Look at the last 100 payments...

sample_trans_green_wide = widen_trans_per_account(sample_trans_green, GREEN, WIDTH)

print sample_trans_green_wide.shape
print
print sample_trans_green_wide.describe()
print
print sample_trans_green_wide[:3]
print

sample_trans_yellow_wide = widen_trans_per_account(sample_trans_yellow, YELLOW, WIDTH)

print sample_trans_yellow_wide.shape
print
print sample_trans_yellow_wide.describe()
print
print sample_trans_yellow_wide[:3]
print

sample_trans_red_wide = widen_trans_per_account(sample_trans_red, RED, WIDTH)

print sample_trans_red_wide.shape
print
print sample_trans_red_wide.describe()
print
print sample_trans_red_wide[:3]
print

sample_trans_wide = pd.concat([sample_trans_green_wide, sample_trans_yellow_wide, sample_trans_red_wide])
sample_trans_wide = sample_trans_wide.sort_values('TRANSACTION_DATE')
sample_trans_wide = sample_trans_wide.drop('TRANSACTION_DATE', 1)

print sample_trans_wide.describe()
print
print sample_trans_wide[:3]
print

shift_encoder = preprocessing.LabelEncoder()
shift_encoder.fit(sample_trans_wide.shift_1.drop_duplicates())

use_case_encoder = preprocessing.LabelEncoder()
use_case_encoder.fit(sample_trans_wide.USE_CASE.drop_duplicates())

sample_trans_wide_enc = pd.DataFrame(index=sample_trans_wide.index)

sample_trans_wide_enc['USE_CASE'] = use_case_encoder.transform(sample_trans_wide.USE_CASE)

for i in range(WIDTH):
  sample_trans_wide_enc['shift_' + str(i) + '_enc'] = shift_encoder.transform(sample_trans_wide['shift_' + str(i)])

print sample_trans_wide_enc[:5]
print 

X = sample_trans_wide_enc[sample_trans_wide_enc.columns[1:]]
print X[:5]
print 

y = sample_trans_wide_enc[sample_trans_wide_enc.columns[0]]
print y[:5]
print

X_enc = pd.DataFrame()
for i in range(WIDTH):
  enc = preprocessing.OneHotEncoder()
  enc.fit(X['shift_' + str(i) + '_enc'].values.reshape((-1,1)))
  X_enc = pd.concat([X_enc, pd.DataFrame(enc.transform(X['shift_' + str(i) + '_enc'].values.reshape((-1,1))).toarray())], axis=1)

X_enc.columns = [i for i in range(len(X_enc.columns))]
print X_enc[:5]
print

X_split = int(len(X_enc) * 0.8)
X_train = X_enc[:X_split]
X_test = X_enc[X_split:]

y_split = int(len(y) * 0.8)
y_train = y[:y_split]
y_test = y[y_split:]

print X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
classifier = RandomForestClassifier(n_estimators = 250)
classifier.fit(X_train, y_train)

print accuracy_score(y_train, classifier.predict(X_train))
print accuracy_score(y_test, classifier.predict(X_test))
print

y_pred = classifier.predict(X_test)
print metrics.classification_report(y_test, y_pred)
print
print metrics.confusion_matrix(y_test, y_pred)

In [ ]:
def model(X, y):
  layers = skflow.ops.dnn(X, [1600], keep_prob=0.5)
  return skflow.models.logistic_regression(layers, y)

classifier = skflow.TensorFlowEstimator(
  model_fn=model, n_classes=3, steps=10000, learning_rate=0.003, optimizer='Adam')
classifier.fit(X_train, y_train)

print accuracy_score(y_train, classifier.predict(X_train))
print accuracy_score(y_test, classifier.predict(X_test))
print

y_pred = classifier.predict(X_test)
print metrics.classification_report(y_test, y_pred)
print
print metrics.confusion_matrix(y_test, y_pred)